In [1]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-3-5-haiku-latest"

In [2]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [3]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    response = chat(messages, stop_sequences=["```"])
    return json.loads(response)

In [4]:
dataset = generate_dataset()

dataset

[{'task': 'Create a Python function to extract the AWS region from an EC2 instance ARN'},
 {'task': 'Write a JSON schema to validate an AWS IAM policy document'},
 {'task': 'Develop a regular expression to match valid AWS resource names (containing only lowercase letters, numbers, and hyphens)'}]

In [5]:
with open("dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [6]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
    Please solve the following task:

    {test_case['task']}
    """

    messages = []
    add_user_message(messages, prompt)
    output = chat(messages)
    return output

In [7]:
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)

    # TODO - Grading
    score = 10

    return {
        "output": output,
        "test_case": test_case,
        "score": score,
    }


In [8]:
def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""

    results = []
    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    return results

In [9]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

In [10]:
results

[{'output': 'Here\'s a Python function that extracts the AWS region from an EC2 instance ARN:\n\n```python\ndef extract_region_from_arn(arn):\n    """\n    Extract the AWS region from an EC2 instance ARN.\n    \n    Args:\n        arn (str): The full AWS EC2 instance ARN\n    \n    Returns:\n        str: The AWS region extracted from the ARN\n        None: If the ARN is invalid or region cannot be extracted\n    """\n    # Validate ARN format\n    if not arn or not isinstance(arn, str):\n        return None\n    \n    # Split the ARN into its components\n    # Typical ARN format: \n    # arn:aws:ec2:region:account-id:instance/instance-id\n    arn_parts = arn.split(\':\')\n    \n    # Check if ARN has the expected number of parts\n    if len(arn_parts) < 4:\n        return None\n    \n    # The region is typically the 4th part (index 3)\n    region = arn_parts[3]\n    \n    # Additional validation\n    if not region:\n        return None\n    \n    return region\n\n# Example usage and t